# TSP问题优化TSP Optimization

In [15]:
import torch
import torch.nn as nn
import time
#argparse：用于解析命令行参数，方便用户通过命令行传递超参数。
import argparse
from tqdm import tqdm

import os
import datetime

# Categorical：来自 PyTorch 的分布类，用于实现分类变量的概率分布，在强化学习或随机采样中常用。
from torch.distributions.categorical import Categorical

# %matplotlib inline：Jupyter Notebook 特有的魔法命令，使绘制的图表直接显示在笔记本中。
# set_matplotlib_formats：设置 Matplotlib 输出格式为高分辨率 PNG 和 PDF。
# visualization 
%matplotlib inline
from IPython.display import set_matplotlib_formats, clear_output
set_matplotlib_formats('png2x','pdf')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from math import sqrt

# networkx：用于创建、操作和研究复杂网络结构的图形库，可以辅助 TSP 问题的可视化。
# scipy.spatial.distance.pdist 和 squareform：计算城市之间的距离矩阵，对于解决 TSP 问题非常重要。
import networkx as nx
from scipy.spatial.distance import pdist, squareform

import warnings
warnings.filterwarnings("ignore", category=UserWarning)


C:\Users\Pengji Jin\AppData\Local\Temp\ipykernel_1148\2429186550.py:19: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('png2x','pdf')


设置 PyTorch 使用的计算设备（CPU 或 GPU），并且可以指定特定的 GPU。

In [16]:

device = torch.device("cpu"); gpu_id = -1 # select CPU

gpu_id = '0' # select a single GPU  
#gpu_id = '2,3' # select multiple GPUs  
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)  
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('GPU name: {:s}, gpu_id: {:s}'.format(torch.cuda.get_device_name(0),gpu_id))   
    
print(device)

GPU name: NVIDIA GeForce RTX 3070 Ti Laptop GPU, gpu_id: 0
cuda


超参数设置Hyperparameter

In [17]:
#定义了一个 DotDict 类，并使用它来创建一个配置对象 args，该对象用于存储训练 TSP（旅行商问题）模型时的各种超参数和设置
class DotDict(dict):
    def __init__(self, **kwds):
        self.update(kwds)
        self.__dict__ = self
# nb_nodes：TSP 问题中的城市数量。这里最后被设置为 50，表示解决的是 TSP50 问题。注释掉了其他选项（如 20 和 100），但可以根据需要取消注释以改变问题规模。
# bsz：批次大小，即每个训练步骤处理的样本数量。这里设置为 512。
# dim_emb：嵌入维度，通常用于表示节点特征的空间维度。
# dim_ff：前馈网络（Feed-forward Network）的维度，常用于 Transformer 模型中的中间层。
# dim_input_nodes：每个节点输入特征的数量。对于 TSP 问题，通常是二维坐标 (x, y)，所以设置为 2。
# nb_layers_encoder 和 nb_layers_decoder：编码器和解码器的层数，分别设置为 6 和 2。
# nb_heads：多头注意力机制中的头数，设置为 8。
# nb_epochs：训练的总周期数，设置为 10000。
# nb_batch_per_epoch：每个 epoch 中的批次数量，设置为 2500。
# nb_batch_eval：评估过程中使用的批次数量，设置为 20。
# gpu_id：GPU ID，从之前设置的变量 gpu_id 获取。
# lr：学习率，设置为 1e-4。
# tol：容差值，用于基线更新条件，设置为 1e-3。
# batchnorm：是否使用批量归一化（Batch Normalization）。这里设置为 True，意味着使用批量归一化；注释掉的选项是使用层归一化（Layer Normalization）。
# max_len_PE：位置编码的最大长度，设置为 1000。   
# 1. 节点数量 (nb_nodes)
# 当前设置：args.nb_nodes = 50
# 建议：对于不同的TSP问题规模（如TSP20, TSP50, TSP100），确保你的模型架构和超参数能够适应不同规模的问题。如果你打算处理更大的实例（例如TSP100），可能需要增加模型容量或调整其他参数。
# 2. 批次大小 (bsz)
# 当前设置：args.bsz = 512
# 建议：较大的批次大小可以加速训练并利用GPU的优势，但也可能导致内存不足。尝试找到一个平衡点，既能充分利用硬件资源，又不会超出显存限制。如果遇到内存问题，考虑减小批次大小。
# 3. 嵌入维度 (dim_emb) 和前馈网络维度 (dim_ff)
# 当前设置：args.dim_emb = 128, args.dim_ff = 512
# 建议：这些值看起来是合理的，但对于更复杂的任务或更大规模的数据集，可以尝试增加这两个维度以增强模型表达能力。同时要注意，增大这些值会显著增加计算量和内存需求。
# 4. 输入维度 (dim_input_nodes)
# 当前设置：args.dim_input_nodes = 2
# 建议：保持不变，因为这是由TSP问题本身决定的（每个节点有二维坐标）。
# 5. 编码器层数 (nb_layers_encoder) 和解码器层数 (nb_layers_decoder)
# 当前设置：args.nb_layers_encoder = 6, args.nb_layers_decoder = 2
# 建议：编码器层数通常是解码器层数的两倍或更多，这有助于捕捉输入序列中的复杂模式。你可以根据实验结果调整这两者之间的比例。如果发现模型过拟合，可以尝试减少层数；如果欠拟合，则可以适当增加。
# 6. 注意力头数 (nb_heads)
# 当前设置：args.nb_heads = 8
# 建议：这个值通常是合理的，但也可以尝试不同的头数来观察效果。更多的头数可以让模型学习到更细粒度的信息，但也增加了计算负担。
# 7. 训练轮数 (nb_epochs) 和每轮批次数 (nb_batch_per_epoch)
# 当前设置：args.nb_epochs = 10000, args.nb_batch_per_epoch = 2500
# 建议：如此多的训练轮数可能没有必要，特别是如果你使用早停（early stopping）策略。监控验证集上的性能，并在性能不再提升时停止训练。每轮批次数可以根据数据集大小和可用计算资源进行调整。
# 8. 评估批次数 (nb_batch_eval)
# 当前设置：args.nb_batch_eval = 20
# 建议：保持合理数量的评估批次，以获得对模型性能的稳定估计。如果评估集较大，可以适当增加这个值。
# 9. 学习率 (lr)
# 当前设置：args.lr = 1e-4
# 建议：学习率的选择非常关键。可以从较低的学习率开始，然后通过学习率调度器（如线性衰减、余弦退火等）逐步调整。此外，使用学习率查找器（learning rate finder）可以帮助确定最优初始学习率。
# 10. 容忍度 (tol)
# 当前设置：args.tol = 1e-3
# 建议：这个参数用于判断是否停止训练，具体取决于你使用的优化算法。如果使用早停机制，确保容忍度设置合理，既不过于宽松也不过于严格。
# 11. 归一化 (batchnorm vs layernorm)
# 当前设置：args.batchnorm = True
# 建议：Batch Normalization（BN）适用于批处理大小较大的情况，而 Layer Normalization（LN）则更适合较小的批处理大小或长序列。考虑到Transformer模型的特点，Layer Norm通常表现更好。可以尝试切换到Layer Norm，看看是否能带来性能提升。
# 12. 位置编码最大长度 (max_len_PE)
# 当前设置：args.max_len_PE = 1000
# 建议：这个值应该足够大以覆盖所有可能的序列长度。如果你确实不需要这么大的值，可以适当减小以节省内存。     
args = DotDict()
args.nb_nodes = 20 # TSP20
# args.nb_nodes = 50 # TSP50
# args.nb_nodes = 100 # TSP100
args.bsz = 512 # TSP20 TSP50
args.dim_emb = 128
args.dim_ff = 512
args.dim_input_nodes = 2
args.nb_layers_encoder = 6
args.nb_layers_decoder = 2
args.nb_heads = 8
args.nb_epochs = 10000
args.nb_batch_per_epoch = 2500
args.nb_batch_eval = 20
args.gpu_id = gpu_id
args.lr = 1e-4
args.tol = 1e-3
args.batchnorm = True  # if batchnorm=True  than batch norm is used
#args.batchnorm = False # if batchnorm=False than layer norm is used
args.max_len_PE = 1000

print(args)

{'nb_nodes': 20, 'bsz': 512, 'dim_emb': 128, 'dim_ff': 512, 'dim_input_nodes': 2, 'nb_layers_encoder': 6, 'nb_layers_decoder': 2, 'nb_heads': 8, 'nb_epochs': 10000, 'nb_batch_per_epoch': 2500, 'nb_batch_eval': 20, 'gpu_id': '0', 'lr': 0.0001, 'tol': 0.001, 'batchnorm': True, 'max_len_PE': 1000}


生成并保存测试集generate and save test dataset

In [18]:
# save_1000tsp：布尔变量，决定是否生成并保存新的测试集。这里被设置为 False，意味着不会执行保存操作。
# 生成测试集：
# 如果 save_1000tsp 为 True，则会生成包含 1000 个实例的测试集，每个实例有 args.nb_nodes 个城市，每个城市有两个坐标值（dim_input_nodes=2）。
# 根据 nb_node 决定生成测试集的名字
# 数据在 CPU 上生成，并打印其大小和第一个实例的内容以供检查。
# 保存测试集：
# 创建 data 目录（如果不存在）。
# 根据 args.nb_nodes 的值将测试集保存为对应的文件名（例如 1000tsp20.pkl），以便后续使用。
save_1000tsp = True
# save_1000tsp = False
if save_1000tsp:
    bsz = 1000
    x = torch.rand(bsz, args.nb_nodes, args.dim_input_nodes, device='cpu') 
    print(x.size(),x[0])
    data_dir = os.path.join("data")
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    if args.nb_nodes==20 : torch.save({ 'x': x, }, '{}.pkl'.format(data_dir + "/1000tsp20"))
    if args.nb_nodes==50 : torch.save({ 'x': x, }, '{}.pkl'.format(data_dir + "/1000tsp50"))
    if args.nb_nodes==100 : torch.save({ 'x': x, }, '{}.pkl'.format(data_dir + "/1000tsp100"))

# 尝试加载现有测试集：
# 根据 args.nb_nodes 的值尝试从 data 目录加载相应的测试集文件。
# 如果成功加载，则将数据移动到当前设备（CPU 或 GPU），并记录城市数量 n。
# 如果没有找到已保存的测试集：
# 在 CPU 上重新生成 1000 个实例的测试集。
# 记录城市数量 n。
checkpoint = None
if args.nb_nodes==20 : checkpoint = torch.load("data/1000tsp20.pkl")
if args.nb_nodes==50 : checkpoint = torch.load("data/1000tsp50.pkl")
if args.nb_nodes==100 : checkpoint = torch.load("data/1000tsp100.pkl")
if checkpoint is not None:
    x_1000tsp = checkpoint['x'].to(device)
    n = x_1000tsp.size(1)
    print('nb of nodes :',n)
else:
    x_1000tsp = torch.rand(1000, args.nb_nodes, args.dim_input_nodes, device='cpu')
    n = x_1000tsp.size(1)
    print('nb of nodes :',n)


torch.Size([1000, 20, 2]) tensor([[0.3670, 0.7802],
        [0.2989, 0.9602],
        [0.4519, 0.8380],
        [0.6201, 0.9643],
        [0.5885, 0.6127],
        [0.2668, 0.0044],
        [0.8015, 0.2750],
        [0.7790, 0.3143],
        [0.5775, 0.4547],
        [0.1175, 0.7685],
        [0.1504, 0.3194],
        [0.7096, 0.9206],
        [0.2231, 0.9546],
        [0.4840, 0.7846],
        [0.3182, 0.7066],
        [0.0043, 0.5257],
        [0.1106, 0.8115],
        [0.3645, 0.5307],
        [0.9666, 0.0883],
        [0.5323, 0.7243]])
nb of nodes : 20


路径长度计算compute tour length

In [19]:
def compute_tour_length(x, tour): 
    """
    Compute the length of a batch of tours
    Inputs : x of size (bsz, nb_nodes, 2) batch of tsp tour instances
             tour of size (bsz, nb_nodes) batch of sequences (node indices) of tsp tours
    Output : L of size (bsz,)             batch of lengths of each tsp tour
    """
    bsz = x.shape[0]
    nb_nodes = x.shape[1]#这里从输入张量 x 中提取了批量大小 bsz 和每个实例中的节点数量 nb_nodes。
    arange_vec = torch.arange(bsz, device=x.device)#创建了一个范围向量 arange_vec，用于索引批量中的每一个样本。这有助于在后续代码中高效地选择特定于每个样本的城市坐标。
    first_cities = x[arange_vec, tour[:,0], :] # size(first_cities)=(bsz,2)
    previous_cities = first_cities#选择了每个路径的第一个城市，并将这些城市设为 previous_cities，以便开始计算路径长度。
    L = torch.zeros(bsz, device=x.device)#初始化一个形状为 (bsz,) 的零张量 L，用来累积每条路径的长度。
    with torch.no_grad():
        for i in range(1,nb_nodes):
            current_cities = x[arange_vec, tour[:,i], :] 
            L += torch.sum( (current_cities - previous_cities)**2 , dim=1 )**0.5 # dist(current, previous node) 
            previous_cities = current_cities
        L += torch.sum( (current_cities - first_cities)**2 , dim=1 )**0.5 # dist(last, first node)  
    return L#遍历每个路径上的所有城市对（除了最后一个到第一个城市之间的距离）。对于每一对连续的城市，它计算欧几里得距离并将结果加到 L 中

# 神经网络部分Network

编码器encoder

In [20]:
# nn.Module 是 PyTorch 中所有神经网络模型的基础类。如果你想要创建一个自定义的神经网络，你需要继承 nn.Module 并实现至少一个方法：forward()
class Transformer_encoder_net(nn.Module):
    def __init__(self, nb_layers, dim_emb, nb_heads, dim_ff, batchnorm):
        super(Transformer_encoder_net, self).__init__()
        # 确认嵌入维度 dim_emb 可以被头数 nb_heads 整除
        # 这是因为每个头将处理一部分嵌入维度,输入的嵌入向量会被分割成多个较小的向量，每个向量对应一个“头”。如果 dim_emb 能够被 nb_heads 整除，那么每个头可以均匀地分配到相同大小的子空间进行并行计算。
        assert dim_emb == nb_heads* (dim_emb//nb_heads) 
        # 创建一个包含 nb_layers 个 MultiheadAttention 层的列表。
        # 每个注意力层都接收相同大小的输入，并输出相同大小的结果。
        self.MHA_layers = nn.ModuleList( [nn.MultiheadAttention(dim_emb, nb_heads) for _ in range(nb_layers)] )

        #为每个编码器层创建两个线性变换层：第一个将输入从 dim_emb 映射到更大的维度 dim_ff，第二个再映射回 dim_emb。这些用于实现前馈神经网络部分。
        #nn.Linear和以下的两个都是Pytorch中的模块
        self.linear1_layers = nn.ModuleList( [nn.Linear(dim_emb, dim_ff) for _ in range(nb_layers)] )
        self.linear2_layers = nn.ModuleList( [nn.Linear(dim_ff, dim_emb) for _ in range(nb_layers)] )   
        #根据是否启用了批量归一化，创建相应的归一化层列表。如果未启用，则使用层归一化。
        if batchnorm:
            self.norm1_layers = nn.ModuleList( [nn.BatchNorm1d(dim_emb) for _ in range(nb_layers)] )
            self.norm2_layers = nn.ModuleList( [nn.BatchNorm1d(dim_emb) for _ in range(nb_layers)] )
        else:
            self.norm1_layers = nn.ModuleList( [nn.LayerNorm(dim_emb) for _ in range(nb_layers)] )
            self.norm2_layers = nn.ModuleList( [nn.LayerNorm(dim_emb) for _ in range(nb_layers)] )
        #保存参数名
        self.nb_layers = nb_layers
        self.nb_heads = nb_heads
        self.batchnorm = batchnorm

    def forward(self, h):
        # PyTorch nn.MultiheadAttention requires input size (seq_len, bsz, dim_emb) 
        # size(h)=(nb_nodes, bsz, dim_emb)  形状为 (bsz, nb_nodes+1, dim_emb) 的张量，表示一批大小为 bsz 的节点（城市）嵌入，每个节点有 dim_emb 维的特征。
        # score为注意力分数，表示节点间的关系，h为输入数据经过多层编码器处理后得到的更新节点嵌入
        # 将张量的0,1项交换
        h = h.transpose(0,1) 
        # L layers接下来是遍历每一层编码器的循环：
        for i in range(self.nb_layers):
            #多头注意力机制 (MHA)
            h_rc = h # residual connection, size(h_rc)=(nb_nodes, bsz, dim_emb)残差连接 (residual connection)：在应用多头注意力之前保存当前状态 h_rc，以便之后可以添加回输出中。
            #调用多头注意力层（已在前面定义） self.MHA_layers[i]，传入相同的查询、键和值（即 h），得到更新后的 h 和注意力分数 score。
            h, score = self.MHA_layers[i](h, h, h) # size(h)=(nb_nodes, bsz, dim_emb), size(score)=(bsz, nb_nodes, nb_nodes)
            # add residual connection
            h = h_rc + h # size(h)=(nb_nodes, bsz, dim_emb)将原始输入 h_rc 添加到多头注意力的输出 h 上，实现跳跃连接，有助于缓解深度网络中的梯度消失问题。
            #根据是否启用了批量归一化（Batch Normalization），选择不同的归一化方式
            #如果使用批量归一化，则需要调整张量形状以匹配 BatchNorm1d 的期望输入格式 (bsz, dim, seq_len)，应用归一化后再恢复原形状。
            #否则直接应用层归一化（Layer Normalization），保持原有形状不变。
            #归一化
            if self.batchnorm:
                # Pytorch nn.BatchNorm1d requires input size (bsz, dim, seq_len)
                h = h.permute(1,2,0).contiguous() # size(h)=(bsz, dim_emb, nb_nodes)
                h = self.norm1_layers[i](h)       # size(h)=(bsz, dim_emb, nb_nodes)
                h = h.permute(2,0,1).contiguous() # size(h)=(nb_nodes, bsz, dim_emb)
            else:
                h = self.norm1_layers[i](h)       # size(h)=(nb_nodes, bsz, dim_emb) 
            # feedforward前馈神经网络 (FFN)
            # 再次使用残差连接保存当前状态 h_rc。
            # 应用ReLU激活函数后的线性变换，然后再次进行线性变换以返回原始嵌入维度。
            # 添加残差连接，确保信息流动的同时引入非线性。
            h_rc = h # residual connection
            h = self.linear2_layers[i](torch.relu(self.linear1_layers[i](h)))
            h = h_rc + h # size(h)=(nb_nodes, bsz, dim_emb)
            #第二次归一化
            if self.batchnorm:
                h = h.permute(1,2,0).contiguous() # size(h)=(bsz, dim_emb, nb_nodes)
                h = self.norm2_layers[i](h)       # size(h)=(bsz, dim_emb, nb_nodes)
                h = h.permute(2,0,1).contiguous() # size(h)=(nb_nodes, bsz, dim_emb)
            else:
                h = self.norm2_layers[i](h) # size(h)=(nb_nodes, bsz, dim_emb)
        # Transpose h在所有编码器层处理完毕后，将张量重新排列回初始形状 (bsz, nb_nodes, dim_emb)，以便后续处理或输出。
        h = h.transpose(0,1) # size(h)=(bsz, nb_nodes, dim_emb)
        return h, score

自定义的多头注意力（Multi-Head Attention, MHA）机制，旨在避免在每次调用时重新计算所有的线性变换。

In [21]:
# 输入
# Q: 形状为 (bsz, dim_emb, 1) 的张量，表示一批查询向量。
# K: 形状为 (bsz, dim_emb, nb_nodes+1) 的张量，表示一批键向量。
# V: 形状为 (bsz, dim_emb, nb_nodes+1) 的张量，表示一批值向量。
# nb_heads: 多头注意力中的头数。
# mask (可选): 形状为 (bsz, nb_nodes+1) 的张量，用于遮蔽某些节点，例如已经访问过的城市。
# clip_value (可选): 一个标量，用于剪切注意力权重。
# 输出
# attn_output: 形状为 (bsz, 1, dim_emb) 的张量，表示一批注意力输出向量。
# attn_weights: 形状为 (bsz, 1, nb_nodes+1) 的张量，表示一批注意力权重。
def myMHA(Q, K, V, nb_heads, mask=None, clip_value=None):
    """
    Compute multi-head attention (MHA) given a query Q, key K, value V and attention mask :
      h = Concat_{k=1}^nb_heads softmax(Q_k^T.K_k).V_k 
    Note : We did not use nn.MultiheadAttention to avoid re-computing all linear transformations at each call.
    Inputs : Q of size (bsz, dim_emb, 1)                batch of queries
             K of size (bsz, dim_emb, nb_nodes+1)       batch of keys
             V of size (bsz, dim_emb, nb_nodes+1)       batch of values
             mask of size (bsz, nb_nodes+1)             batch of masks of visited cities
             clip_value is a scalar 
    Outputs : attn_output of size (bsz, 1, dim_emb)     batch of attention vectors
              attn_weights of size (bsz, 1, nb_nodes+1) batch of attention weights
    """
    # 这里从 K 中提取了批量大小 bsz、节点数量 nb_nodes 和嵌入维度 emd_dim。同时，注释中提到 dim_emb 必须可以被 nb_heads 整除，以确保每个头处理相同大小的子空间。
    bsz, nb_nodes, emd_dim = K.size() #  dim_emb must be divisable by nb_heads
    # 当 nb_heads > 1 ,即有多个头时，需要将 Q、K 和 V 张量重新排列并分割成多个头,确保了每个头能够独立地处理不同部分的嵌入向量，同时也保证了形状的一致性。
    if nb_heads>1:
        # PyTorch view requires contiguous dimensions for correct reshaping
        Q = Q.transpose(1,2).contiguous() # size(Q)=(bsz, dim_emb, 1)
        Q = Q.view(bsz*nb_heads, emd_dim//nb_heads, 1) # size(Q)=(bsz*nb_heads, dim_emb//nb_heads, 1)
        Q = Q.transpose(1,2).contiguous() # size(Q)=(bsz*nb_heads, 1, dim_emb//nb_heads)
        K = K.transpose(1,2).contiguous() # size(K)=(bsz, dim_emb, nb_nodes+1)
        K = K.view(bsz*nb_heads, emd_dim//nb_heads, nb_nodes) # size(K)=(bsz*nb_heads, dim_emb//nb_heads, nb_nodes+1)
        K = K.transpose(1,2).contiguous() # size(K)=(bsz*nb_heads, nb_nodes+1, dim_emb//nb_heads)
        V = V.transpose(1,2).contiguous() # size(V)=(bsz, dim_emb, nb_nodes+1)
        V = V.view(bsz*nb_heads, emd_dim//nb_heads, nb_nodes) # size(V)=(bsz*nb_heads, dim_emb//nb_heads, nb_nodes+1)
        V = V.transpose(1,2).contiguous() # size(V)=(bsz*nb_heads, nb_nodes+1, dim_emb//nb_heads)
    #批矩阵乘法 (torch.bmm) 来计算查询与键之间的点积，然后除以根号下的查询维度，这一步是为了缩放点积结果，防止梯度消失或爆炸问题。
    attn_weights = torch.bmm(Q, K.transpose(1,2))/ Q.size(-1)**0.5 # size(attn_weights)=(bsz*nb_heads, 1, nb_nodes+1)
    #剪切：通过 tanh 函数限制注意力权重的范围，并乘以 clip_value。
    #遮蔽：对于已经被访问过的节点，应用极大的负值（如 -1e9），使得它们在 softmax 后的概率接近于零。
    if clip_value is not None:
        attn_weights = clip_value * torch.tanh(attn_weights)
    if mask is not None:
        if nb_heads>1:
            mask = torch.repeat_interleave(mask, repeats=nb_heads, dim=0) # size(mask)=(bsz*nb_heads, nb_nodes+1)
        #attn_weights = attn_weights.masked_fill(mask.unsqueeze(1), float('-inf')) # size(attn_weights)=(bsz*nb_heads, 1, nb_nodes+1)
        attn_weights = attn_weights.masked_fill(mask.unsqueeze(1), float('-1e9')) # size(attn_weights)=(bsz*nb_heads, 1, nb_nodes+1)
    attn_weights = torch.softmax(attn_weights, dim=-1) # size(attn_weights)=(bsz*nb_heads, 1, nb_nodes+1)应用 softmax 函数，将注意力权重转换为概率分布。
    attn_output = torch.bmm(attn_weights, V) # size(attn_output)=(bsz*nb_heads, 1, dim_emb//nb_heads)再次使用批矩阵乘法来计算加权和，得到最终的注意力输出。
    if nb_heads>1:
        attn_output = attn_output.transpose(1,2).contiguous() # size(attn_output)=(bsz*nb_heads, dim_emb//nb_heads, 1)
        attn_output = attn_output.view(bsz, emd_dim, 1) # size(attn_output)=(bsz, dim_emb, 1)
        attn_output = attn_output.transpose(1,2).contiguous() # size(attn_output)=(bsz, 1, dim_emb)
        attn_weights = attn_weights.view(bsz, nb_heads, 1, nb_nodes) # size(attn_weights)=(bsz, nb_heads, 1, nb_nodes+1)
        attn_weights = attn_weights.mean(dim=1) # mean over the heads, size(attn_weights)=(bsz, 1, nb_nodes+1)
    return attn_output, attn_weights

自回归解码器层，主要用于生成序列数据的任务

In [22]:
class AutoRegressiveDecoderLayer(nn.Module):
    """
    Single decoder layer based on self-attention and query-attention
    Inputs :  
      h_t of size      (bsz, 1, dim_emb)          batch of input queries
      K_att of size    (bsz, nb_nodes+1, dim_emb) batch of query-attention keys
      V_att of size    (bsz, nb_nodes+1, dim_emb) batch of query-attention values
      mask of size     (bsz, nb_nodes+1)          batch of masks of visited cities
    Output :  
      h_t of size (bsz, nb_nodes+1)               batch of transformed queries
    """
    #
    def __init__(self, dim_emb, nb_heads):
        super(AutoRegressiveDecoderLayer, self).__init__()
        self.dim_emb = dim_emb# 嵌入维度
        self.nb_heads = nb_heads# 注意力头的数量

         # 自注意力机制中的线性变换层
        self.Wq_selfatt = nn.Linear(dim_emb, dim_emb)
        self.Wk_selfatt = nn.Linear(dim_emb, dim_emb)
        self.Wv_selfatt = nn.Linear(dim_emb, dim_emb)
        self.W0_selfatt = nn.Linear(dim_emb, dim_emb)

        # 查询注意力机制中的线性变换层
        self.W0_att = nn.Linear(dim_emb, dim_emb)
        self.Wq_att = nn.Linear(dim_emb, dim_emb)
        self.W1_MLP = nn.Linear(dim_emb, dim_emb)
        self.W2_MLP = nn.Linear(dim_emb, dim_emb)

         # 批规范化层
        self.BN_selfatt = nn.LayerNorm(dim_emb)
        self.BN_att = nn.LayerNorm(dim_emb)
        self.BN_MLP = nn.LayerNorm(dim_emb)
        
         # 初始化累积的自注意力键和值为 None
        self.K_sa = None
        self.V_sa = None

    #将 self.K_sa 和 self.V_sa 设置为 None，用于清除之前的累积键和值，这通常是在处理新序列时调用的。
    def reset_selfatt_keys_values(self):
        self.K_sa = None
        self.V_sa = None
        

    #h_t扮演query的角色，前向传播函数
    def forward(self, h_t, K_att, V_att, mask):
        bsz = h_t.size(0) # 获取批次大小
        h_t = h_t.view(bsz,1,self.dim_emb) # size(h_t)=(bsz, 1, dim_emb)调整 h_t 的形状以适应后续操作
        # embed the query for self-attention 自注意力机制
        q_sa = self.Wq_selfatt(h_t) # size(q_sa)=(bsz, 1, dim_emb)
        k_sa = self.Wk_selfatt(h_t) # size(k_sa)=(bsz, 1, dim_emb)
        v_sa = self.Wv_selfatt(h_t) # size(v_sa)=(bsz, 1, dim_emb)
        # concatenate the new self-attention key and value to the previous keys and values# 累积自注意力键和值
        #检查是否为第一个时间步，是的话初始化累积的键和值，不是的话使用 torch.cat 函数沿指定维度（这里是 dim=1）拼接现有的累积键/值与当前时间步的键/值。
        if self.K_sa is None:
            self.K_sa = k_sa # size(self.K_sa)=(bsz, 1, dim_emb)
            self.V_sa = v_sa # size(self.V_sa)=(bsz, 1, dim_emb)
        else:
            self.K_sa = torch.cat([self.K_sa, k_sa], dim=1)
            self.V_sa = torch.cat([self.V_sa, v_sa], dim=1)

        # compute self-attention between nodes in the partial tour  计算自注意力并更新 h_t
        h_t = h_t + self.W0_selfatt( myMHA(q_sa, self.K_sa, self.V_sa, self.nb_heads)[0] ) # size(h_t)=(bsz, 1, dim_emb)
        h_t = self.BN_selfatt(h_t.squeeze()) # size(h_t)=(bsz, dim_emb)# 应用批规范化
        h_t = h_t.view(bsz, 1, self.dim_emb) # size(h_t)=(bsz, 1, dim_emb)# 调整形状
        # compute attention between self-attention nodes and encoding nodes in the partial tour (translation process)
        ## 查询注意力机制
        q_a = self.Wq_att(h_t) # size(q_a)=(bsz, 1, dim_emb)
        h_t = h_t + self.W0_att( myMHA(q_a, K_att, V_att, self.nb_heads, mask)[0] ) # size(h_t)=(bsz, 1, dim_emb)更新 h_t
        h_t = self.BN_att(h_t.squeeze()) # size(h_t)=(bsz, dim_emb)
        h_t = h_t.view(bsz, 1, self.dim_emb) # size(h_t)=(bsz, 1, dim_emb)
        # MLP # MLP 层
        h_t = h_t + self.W2_MLP(torch.relu(self.W1_MLP(h_t)))
        h_t = self.BN_MLP(h_t.squeeze(1)) # size(h_t)=(bsz, dim_emb)
        return h_t
        

解码器层

In [23]:
class Transformer_decoder_net(nn.Module): 
    """
    Decoder network based on self-attention and query-attention transformers
    Inputs :  
      h_t of size      (bsz, 1, dim_emb)                            batch of input queries
      K_att of size    (bsz, nb_nodes+1, dim_emb*nb_layers_decoder) batch of query-attention keys for all decoding layers
      V_att of size    (bsz, nb_nodes+1, dim_emb*nb_layers_decoder) batch of query-attention values for all decoding layers
      mask of size     (bsz, nb_nodes+1)                            batch of masks of visited cities
    Output :  
      prob_next_node of size (bsz, nb_nodes+1)                      batch of probabilities of next node
    """
    #类初始化 (__init__)，self是类的第一个参数，后面三个是类的实例属性，也就是主要看后三个
    def __init__(self, dim_emb, nb_heads, nb_layers_decoder):
        super(Transformer_decoder_net, self).__init__()
        self.dim_emb = dim_emb # 嵌入维度，每个时间步输入和输出的特征维度
        self.nb_heads = nb_heads # 注意力头的数量
        self.nb_layers_decoder = nb_layers_decoder # 解码器层数
        # AutoRegressiveDecoderLayer 层是 Transformer_decoder_net 中的一个关键组件，
        # 它实现了基于自注意力（self-attention）和查询注意力（query-attention）机制的解码器层。
        # 这种类型的解码器层特别适用于自动回归（auto-regressive）模型，其中每个时间步的输出依赖于之前所有时间步的输出。
        self.decoder_layers = nn.ModuleList( [AutoRegressiveDecoderLayer(dim_emb, nb_heads) for _ in range(nb_layers_decoder-1)] )# 创建多个 AutoRegressiveDecoderLayer 层
        # 最终一层的线性变换层
        self.Wq_final = nn.Linear(dim_emb, dim_emb)
        
    # Reset to None self-attention keys and values when decoding starts 
    #重置累积的自注意力键和值 (reset_selfatt_keys_values)，通常在开始处理新序列时调用
    def reset_selfatt_keys_values(self): 
        for l in range(self.nb_layers_decoder-1):
            self.decoder_layers[l].reset_selfatt_keys_values()
            
    #前向传播 (forward)
    def forward(self, h_t, K_att, V_att, mask):
        for l in range(self.nb_layers_decoder):
            K_att_l = K_att[:,:,l*self.dim_emb:(l+1)*self.dim_emb].contiguous()  # size(K_att_l)=(bsz, nb_nodes+1, dim_emb) # 提取当前层对应的K_att部分
            V_att_l = V_att[:,:,l*self.dim_emb:(l+1)*self.dim_emb].contiguous()  # size(V_att_l)=(bsz, nb_nodes+1, dim_emb) # 提取当前层对应的V_att部分
            if l<self.nb_layers_decoder-1: # decoder layers with multiple heads (intermediate layers) # 对于中间层，不是最后一层，则调用相应的 AutoRegressiveDecoderLayer 进行处理，并更新 h_t
                h_t = self.decoder_layers[l](h_t, K_att_l, V_att_l, mask)
            else: # decoder layers with single head (final layer)# 对于最后一层，使用 Wq_final 线性变换来生成最终的查询 q_final，然后通过 myMHA 函数计算注意力权重 attn_weights，这里使用单头注意力机制。
                q_final = self.Wq_final(h_t)
                bsz = h_t.size(0)
                q_final = q_final.view(bsz, 1, self.dim_emb)
                attn_weights = myMHA(q_final, K_att_l, V_att_l, 1, mask, 10)[1] 
        prob_next_node = attn_weights.squeeze(1) #attn_weights 被挤压成形状 (bsz, nb_nodes+1) 的张量，表示下一个节点的概率分布 prob_next_node。
        return prob_next_node


位置编码,新论文删除了来轻量化模型

In [24]:
#生成标准的 Transformer 位置编码，详见transformer的ppt
def generate_positional_encoding(d_model, max_len):
    """
    Create standard transformer PEs.
    Inputs :  
      d_model is a scalar correspoding to the hidden dimension
      max_len is the maximum length of the sequence
    Output :  
      pe of size (max_len, d_model), where d_model=dim_emb, max_len=1000
    """
    pe = torch.zeros(max_len, d_model) # 初始化位置编码矩阵   
    # 计算位置索引，使用 torch.arange 创建一个从 0 到 max_len-1 的浮点数张量，并通过 unsqueeze(1) 将其转换为二维张量，使得它可以与后续的 div_term 进行广播相乘。
    position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1) 
    # 计算除法因子 (div_term)，公式
    div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))
    #对于所有偶数维度（0::2），使用正弦函数填充位置编码。
    #对于所有奇数维度（1::2），使用余弦函数填充位置编码。
    pe[:,0::2] = torch.sin(position * div_term)
    pe[:,1::2] = torch.cos(position * div_term)
    return pe

结合TSP问题，结合了编码器-解码器架构和自回归解码过程In the context of the TSP problem, it integrates an encoder-decoder architecture with an autoregressive decoding process

In [25]:
class TSP_net(nn.Module): 
    """
    The TSP network is composed of two steps :
      Step 1. Encoder step : Take a set of 2D points representing a fully connected graph 
                             and encode the set with self-transformer.
      Step 2. Decoder step : Build the TSP tour recursively/autoregressively, 
                             i.e. one node at a time, with a self-transformer and query-transformer. 
    Inputs : 
      x of size (bsz, nb_nodes, dim_emb) Euclidian coordinates of the nodes/cities
      deterministic is a boolean : If True the salesman will chose the city with highest probability. 
                                   If False the salesman will chose the city with Bernouilli sampling.
    Outputs : 
      tours of size (bsz, nb_nodes) : batch of tours, i.e. sequences of ordered cities 
                                      tours[b,t] contains the idx of the city visited at step t in batch b
      sumLogProbOfActions of size (bsz,) : batch of sum_t log prob( pi_t | pi_(t-1),...,pi_0 )
    """
    # dim_input_nodes: 输入节点的维度，例如每个城市坐标的维度。
    # dim_emb: 嵌入维度，所有内部表示的统一维度。
    # dim_ff: 前馈神经网络中的隐藏层维度。
    # nb_layers_encoder: 编码器中 Transformer 层的数量。
    # nb_layers_decoder: 解码器中 Transformer 层的数量。
    # nb_heads: 注意力机制中的头数量。
    # max_len_PE: 位置编码的最大长度。
    # batchnorm: 是否使用批规范化，默认为 True。
    def __init__(self, dim_input_nodes, dim_emb, dim_ff, nb_layers_encoder, nb_layers_decoder, nb_heads, max_len_PE,
                 batchnorm=True):
        super(TSP_net, self).__init__()
        
        self.dim_emb = dim_emb
        
        # input embedding layer
        self.input_emb = nn.Linear(dim_input_nodes, dim_emb)
        
        # encoder layer从编码层输入
        self.encoder = Transformer_encoder_net(nb_layers_encoder, dim_emb, nb_heads, dim_ff, batchnorm)
        
        # vector to start decoding 定义一个可训练的参数，作为解码开始时的占位符，形状为 (dim_emb,)
        self.start_placeholder = nn.Parameter(torch.randn(dim_emb))
        
        # decoder layer
        self.decoder = Transformer_decoder_net(dim_emb, nb_heads, nb_layers_decoder)
        self.WK_att_decoder = nn.Linear(dim_emb, nb_layers_decoder* dim_emb) 
        self.WV_att_decoder = nn.Linear(dim_emb, nb_layers_decoder* dim_emb) 
        self.PE = generate_positional_encoding(dim_emb, max_len_PE)        
        
    #前向传播
    # 输入：
    # x: 形状为 (bsz, nb_nodes, dim_emb) 的张量，表示一批城市的欧几里得坐标。
    # deterministic: 决定选择下一个城市的方式——确定性（最高概率）或随机采样（伯努利抽样）。
    # 输出：
    # tours: 形状为 (bsz, nb_nodes) 的张量，表示一批访问顺序的城市索引。
    # sumLogProbOfActions: 形状为 (bsz,) 的张量，表示每个样本的动作序列的累积对数概率。
    def forward(self, x, deterministic=False):

        # some parameters获取批量大小 bsz 和节点数量 nb_nodes。
        #创建从 0 到 bsz-1 的索引数组 zero_to_bsz，用于后续操作。
        bsz = x.shape[0]
        nb_nodes = x.shape[1]
        zero_to_bsz = torch.arange(bsz, device=x.device) # [0,1,...,bsz-1]

        # input embedding layer 用self.input_emb 对输入 x 进行线性变换，得到形状为 (bsz, nb_nodes, dim_emb) 的嵌入表示 h。
        h = self.input_emb(x) # size(h)=(bsz, nb_nodes, dim_emb)
        
        # concat the nodes and the input placeholder that starts the decoding
        # 将解码起始占位符与 h 拼接，形成形状为 (bsz, nb_nodes+1, dim_emb) 的新张量。这允许解码器知道序列的起点。
        h = torch.cat([h, self.start_placeholder.repeat(bsz, 1, 1)], dim=1) # size(start_placeholder)=(bsz, nb_nodes+1, dim_emb)
        
        # encoder layer
        #通过 self.encoder 对拼接后的张量 h 进行编码，得到编码后的表示 h_encoder。编码器会根据上下文信息更新每个节点的表示。
        h_encoder, _ = self.encoder(h) # size(h)=(bsz, nb_nodes+1, dim_emb)

        # list that will contain Long tensors of shape (bsz,) that gives the idx of the cities chosen at time t
        # 这是一个空列表，用于保存一系列形状为 (bsz,) 的长整型张量（LongTensor），其中 bsz 表示批次大小。
        # 内容：每个张量代表在特定时间步 t 选择的城市索引。也就是说，在解码过程中每一步所选择的城市的索引会被添加到这个列表中。
        # 最终结果：当解码完成时，tours 将包含一个长度为 nb_nodes 的列表，每个元素是一个形状为 (bsz,) 的张量，表示批次中每个样本在该时间步选择的城市索引。
        tours = []

        # list that will contain Float tensors of shape (bsz,) that gives the neg log probs of the choices made at time t
        # 类型：这也是一个空列表，但用于保存一系列形状为 (bsz,) 的浮点型张量（FloatTensor）。
        # 内容：每个张量代表在特定时间步 t 所做选择的负对数概率（negative log probabilities）。这里使用负对数概率是因为它在优化过程中更方便处理，并且通常用于计算损失函数。
        # 最终结果：当解码完成时，sumLogProbOfActions 将包含一个长度为 nb_nodes 的列表，每个元素是一个形状为 (bsz,) 的张量，表示批次中每个样本在该时间步选择动作的负对数概率。
        sumLogProbOfActions = []

        # key and value for decoder 准备解码器的键和值   
        K_att_decoder = self.WK_att_decoder(h_encoder) # size(K_att)=(bsz, nb_nodes+1, dim_emb*nb_layers_decoder)
        V_att_decoder = self.WV_att_decoder(h_encoder) # size(V_att)=(bsz, nb_nodes+1, dim_emb*nb_layers_decoder)
        
        # input placeholder that starts the decoding
        # 将位置编码张量移动到与输入张量 x 相同的设备上（例如，CPU 或 GPU）。在 PyTorch 中，确保所有参与计算的张量位于同一设备上是非常重要的，以避免运行时错误并确保高效的计算。
        self.PE = self.PE.to(x.device)
        idx_start_placeholder = torch.Tensor([nb_nodes]).long().repeat(bsz).to(x.device)

        #设置解码起始占位符 h_start，它由编码后的最后一个元素加上位置编码的第一项构成。这是因为在拼接时，起始占位符被添加到了末尾。
        #因为占位符在末尾，所以位置编码第一项也放末尾
        h_start = h_encoder[zero_to_bsz, idx_start_placeholder, :] + self.PE[0].repeat(bsz,1) # size(h_start)=(bsz, dim_emb)
        
        # initialize mask of visited cities
        # 初始化已访问节点的掩码 mask_visited_nodes，将起始占位符标记为已访问
        # torch.zeros(...)：创建一个形状为 (bsz, nb_nodes+1) 的全零张量，其中：
        # bsz 是批次大小，表示一批数据中有多少个独立的 TSP 实例。
        # nb_nodes 是每个 TSP 实例中的城市数量。
        # +1 表示额外添加了一个占位符节点（通常是解码开始时的虚拟节点），因此总共有 nb_nodes + 1 个节点。
        # .bool()：将这个全零张量转换为布尔类型的张量，默认值为 False。这意味着在一开始，所有城市都被标记为未访问。
        # device=x.device：确保这个张量被创建在与输入张量 x 相同的设备上（例如 CPU 或 GPU），以保证后续计算可以在同一设备上进行。
        mask_visited_nodes = torch.zeros(bsz, nb_nodes+1, device=x.device).bool() # False布尔值，只有true和false
        # zero_to_bsz：这是一个从 0 到 bsz-1 的索引数组，用来选择每个批次样本对应的行。
        # idx_start_placeholder：这是指定了起始占位符节点的索引。由于我们在编码阶段末尾添加了这个占位符，它的索引是 nb_nodes（即最后一个位置）。这行代码的意思是对于每个批次样本，我们将该批次中对应于起始占位符节点的位置标记为已访问。
        # 赋值操作：将 mask_visited_nodes 中指定位置的值设置为 True，表示这些位置的城市已经被访问。具体来说，它将每个批次样本中起始占位符节点的位置标记为已访问。
        mask_visited_nodes[zero_to_bsz, idx_start_placeholder] = True
        
        # clear key and val stored in the decoder清除解码器中存储的自注意力键和值，以便开始新的解码过程
        self.decoder.reset_selfatt_keys_values()

        # construct tour recursively
        # 在循环中，对于每一个时间步 t：
        # 计算下一节点的概率分布 prob_next_node。解码器根据当前状态 h_t、编码器提供的键和值 K_att_decoder 和 V_att_decoder 以及已访问节点的掩码 mask_visited_nodes 来预测下一个要访问的城市。
        # 根据 deterministic 参数选择下一个城市索引 idx。如果为真，则选取概率最高的城市；否则，按照概率分布进行采样。
        # 更新当前访问节点的嵌入表示 h_t，并添加相应的时间步位置编码。这一步骤保证了解码器在处理下一个时间步时能考虑到当前的位置信息。
        # 更新路径列表 tours 和动作序列的累积对数概率 sumLogProbOfActions。累积对数概率用于评估生成路径的质量。
        # 更新已访问节点的掩码 mask_visited_nodes，将刚刚选择的城市标记为已访问，防止重复访问。
        h_t = h_start
        for t in range(nb_nodes):
            
            # compute probability over the next node in the tour
            prob_next_node = self.decoder(h_t, K_att_decoder, V_att_decoder, mask_visited_nodes) # size(prob_next_node)=(bsz, nb_nodes+1)
            
            # choose node with highest probability or sample with Bernouilli 
            if deterministic:
                idx = torch.argmax(prob_next_node, dim=1) # size(query)=(bsz,)
            else:
                idx = Categorical(prob_next_node).sample() # size(query)=(bsz,)
            
            # compute logprobs of the action items in the list sumLogProbOfActions   
            ProbOfChoices = prob_next_node[zero_to_bsz, idx] 
            sumLogProbOfActions.append( torch.log(ProbOfChoices) )  # size(query)=(bsz,)

            # update embedding of the current visited node
            h_t = h_encoder[zero_to_bsz, idx, :] # size(h_start)=(bsz, dim_emb)
            h_t = h_t + self.PE[t+1].expand(bsz, self.dim_emb)
            
            # update tour
            tours.append(idx)

            # update masks with visited nodes
            mask_visited_nodes = mask_visited_nodes.clone()
            mask_visited_nodes[zero_to_bsz, idx] = True
            
        # 将 sumLogProbOfActions 和 tours 转换为适当形状的张量，并返回。最终，tours 包含了每一批次中访问城市的顺序，
        # 而 sumLogProbOfActions 则提供了这些路径的累积对数概率，可用于损失计算或其他评估指标。
        # logprob_of_choices = sum_t log prob( pi_t | pi_(t-1),...,pi_0 )
        sumLogProbOfActions = torch.stack(sumLogProbOfActions,dim=1).sum(dim=1) # size(sumLogProbOfActions)=(bsz,)

        # convert the list of nodes into a tensor of shape (bsz,num_cities)
        tours = torch.stack(tours,dim=1) # size(col_index)=(bsz, nb_nodes)
        
        return tours, sumLogProbOfActions
    

EarlyStopping早停减少训练时间

In [26]:
# class EarlyStopping:
#     """Early stops the training if validation loss doesn't improve after a given patience."""
#     def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
#         """
#         Args:
#             patience (int): How long to wait after last time validation loss improved.
#                             Default: 7
#             verbose (bool): If True, prints a message for each validation loss improvement. 
#                             Default: False
#             delta (float): Minimum change in the monitored quantity to qualify as an improvement.
#                             Default: 0
#             path (str): Path for the checkpoint to be saved to.
#                             Default: 'checkpoint.pt'
#         """
#         self.patience = patience
#         self.verbose = verbose
#         self.counter = 0
#         self.best_score = None
#         self.early_stop = False
#         self.val_loss_min = np.Inf
#         self.delta = delta
#         self.path = path

#     def __call__(self, val_loss, model):

#         score = -val_loss

#         if self.best_score is None:
#             self.best_score = score
#             self.save_checkpoint(val_loss, model)
#         elif score < self.best_score + self.delta:
#             self.counter += 1
#             print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
#             if self.counter >= self.patience:
#                 self.early_stop = True
#         else:
#             self.best_score = score
#             self.save_checkpoint(val_loss, model)
#             self.counter = 0

#     def save_checkpoint(self, val_loss, model):
#         '''Saves model when validation loss decrease.'''
#         if self.verbose:
#             print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
#         torch.save(model.state_dict(), self.path)
#         self.val_loss_min = val_loss

初始化Initialization

In [27]:
try: 
    del model_train # remove existing model
    del model_baseline # remove existing model
except:
    pass

# 初始化EarlyStopping对象
if args.nb_nodes>=20 : early_stopping = EarlyStopping(patience=5, verbose=True)
if args.nb_nodes>=50 : early_stopping = EarlyStopping(patience=7, verbose=True)
if args.nb_nodes>=100 : early_stopping = EarlyStopping(patience=10, verbose=True)


# model_train：创建一个用于训练的 TSP 网络实例。
# model_baseline：创建一个基线模型，通常用于比较性能或计算强化学习中的优势函数（advantage function）。
# 参数来源：所有参数都从 args 对象中获取，这通常是通过命令行参数或配置文件传递的超参数集合。
model_train = TSP_net(args.dim_input_nodes, args.dim_emb, args.dim_ff, 
              args.nb_layers_encoder, args.nb_layers_decoder, args.nb_heads, args.max_len_PE,
              batchnorm=args.batchnorm)

model_baseline = TSP_net(args.dim_input_nodes, args.dim_emb, args.dim_ff, 
              args.nb_layers_encoder, args.nb_layers_decoder, args.nb_heads, args.max_len_PE,
              batchnorm=args.batchnorm)

# uncomment these lines if trained with multiple GPUs
# 检查多GPU：使用 torch.cuda.device_count() 检查可用的 GPU 数量。
# 启用 DataParallel：如果有多个 GPU 可用，则使用 nn.DataParallel 包装模型，使它们能够在多个 GPU 上并行运行，从而加速训练过程。
print(torch.cuda.device_count())
if torch.cuda.device_count()>1:
    model_train = nn.DataParallel(model_train)
    model_baseline = nn.DataParallel(model_baseline)
# uncomment these lines if trained with multiple GPUs

# 选择优化器：为 model_train 创建 Adam 优化器，这是一种常用的自适应学习率优化算法。
# 设置学习率：从 args 中获取学习率 lr 并传递给优化器
optimizer = torch.optim.Adam( model_train.parameters() , lr = args.lr ) 

# 指定设备：将两个模型移动到指定的设备上（如 GPU），这里假设 device 已经被正确设置为 'cuda' 或 'cpu'。
# 设置基线模型为评估模式：调用 model_baseline.eval()，告诉模型现在处于评估模式，这样可以禁用 dropout 等训练时特有的行为。
model_train = model_train.to(device)
model_baseline = model_baseline.to(device)
model_baseline.eval()

print(args); print('')

# Logs
# 创建日志目录：如果 logs 目录不存在，则创建它。
# 生成时间戳：获取当前时间的时间戳，用于命名日志文件。
# 打开日志文件：创建一个新的日志文件，并将其打开为写入模式。
# 记录时间和参数：将当前时间和所有的超参数写入日志文件，以便日后参考。
os.system("mkdir logs")
time_stamp=datetime.datetime.now().strftime("%y-%m-%d--%H-%M-%S")
file_name = 'logs'+'/'+time_stamp + "-n{}".format(args.nb_nodes) + "-gpu{}".format(args.gpu_id) + ".txt"
file = open(file_name,"w",1) 
file.write(time_stamp+'\n\n') 
for arg in vars(args):
    file.write(arg)
    hyper_param_val="={}".format(getattr(args, arg))
    file.write(hyper_param_val)
    file.write('\n')
file.write('\n\n') 
plot_performance_train = []
plot_performance_baseline = []
all_strings = []
epoch_ckpt = 0
tot_time_ckpt = 0

1
{'nb_nodes': 20, 'bsz': 512, 'dim_emb': 128, 'dim_ff': 512, 'dim_input_nodes': 2, 'nb_layers_encoder': 6, 'nb_layers_decoder': 2, 'nb_heads': 8, 'nb_epochs': 10000, 'nb_batch_per_epoch': 2500, 'nb_batch_eval': 20, 'gpu_id': '0', 'lr': 0.0001, 'tol': 0.001, 'batchnorm': True, 'max_len_PE': 1000}



训练Training

In [28]:
# # Uncomment these lines to re-start training with saved checkpoint
# checkpoint_file = "checkpoint/checkpoint_21-03-01--17-25-00-n50-gpu0.pkl"
# checkpoint = torch.load(checkpoint_file, map_location=device)
# epoch_ckpt = checkpoint['epoch'] + 1
# tot_time_ckpt = checkpoint['tot_time']
# plot_performance_train = checkpoint['plot_performance_train']
# plot_performance_baseline = checkpoint['plot_performance_baseline']
# model_baseline.load_state_dict(checkpoint['model_baseline'])
# model_train.load_state_dict(checkpoint['model_train'])
# optimizer.load_state_dict(checkpoint['optimizer'])
# print('Re-start training with saved checkpoint file={:s}\n  Checkpoint at epoch= {:d} and time={:.3f}min\n'.format(checkpoint_file,epoch_ckpt-1,tot_time_ckpt/60))
# del checkpoint
# # Uncomment these lines to re-start training with saved checkpoint


###################
# Main training loop 
###################
#记录开始训练的时间戳，用于后续计算总的训练时间。
start_training_time = time.time()

# epoch 循环：遍历指定数量的训练周期（args.nb_epochs），每个周期代表一次完整的数据集遍历。
# 检查点恢复：如果之前有保存的检查点，则从 epoch_ckpt 开始继续训练。
for epoch in range(0,args.nb_epochs):
    
    # re-start training with saved checkpoint
    epoch += epoch_ckpt

    ###################
    # Train model for one epoch
    ###################
    # start：记录当前 epoch 开始的时间。
    # model_train.train()：将 model_train 设置为训练模式，启用如 dropout 等训练特有的行为。
    start = time.time()
    model_train.train() 

    # 生成批次数据：使用 torch.rand 生成一批随机的 TSP 实例，形状为 (bsz, nb_nodes, dim_input_nodes)。
    # 计算路径：
    # 对于 model_train，以非确定性方式（即基于采样）生成路径，并计算累积对数概率 sumLogProbOfActions。
    # 对于 model_baseline，以确定性方式（即选择最高概率的城市）生成路径。
    # 计算路径长度：调用 compute_tour_length 函数计算每条路径的总长度。
    # 损失计算与反向传播：定义损失函数为 (L_train - L_baseline) * sumLogProbOfActions 的平均值，然后执行反向传播和参数更新。
    for step in range(1,args.nb_batch_per_epoch+1):    

        # generate a batch of random TSP instances    
        x = torch.rand(args.bsz, args.nb_nodes, args.dim_input_nodes, device=device) # size(x)=(bsz, nb_nodes, dim_input_nodes) 

        # compute tours for model
        tour_train, sumLogProbOfActions = model_train(x, deterministic=False) # size(tour_train)=(bsz, nb_nodes), size(sumLogProbOfActions)=(bsz)
      
        # compute tours for baseline
        with torch.no_grad():
            tour_baseline, _ = model_baseline(x, deterministic=True)

        # get the lengths of the tours
        L_train = compute_tour_length(x, tour_train) # size(L_train)=(bsz)
        L_baseline = compute_tour_length(x, tour_baseline) # size(L_baseline)=(bsz)
        
        # backprop
        loss = torch.mean( (L_train - L_baseline)* sumLogProbOfActions )
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    # time_one_epoch：记录完成一个 epoch 所需的时间。
    # time_tot：记录总的训练时间，包括之前的检查点时间 tot_time_ckpt。
    time_one_epoch = time.time()-start
    time_tot = time.time()-start_training_time + tot_time_ckpt

        
    ###################
    # Evaluate train model and baseline on 10k random TSP instances
    ###################
    #将 model_train 设置为评估模式，禁用训练特有的行为。
    model_train.eval()
    mean_tour_length_train = 0
    mean_tour_length_baseline = 0
    #在一定数量的随机 TSP 实例上评估 model_train 和 model_baseline 的平均路径长度。
    for step in range(0,args.nb_batch_eval):

        # generate a batch of random tsp instances   
        x = torch.rand(args.bsz, args.nb_nodes, args.dim_input_nodes, device=device) 

        # compute tour for model and baseline
        with torch.no_grad():
            tour_train, _ = model_train(x, deterministic=True)
            tour_baseline, _ = model_baseline(x, deterministic=True)
            
        # get the lengths of the tours
        L_train = compute_tour_length(x, tour_train)
        L_baseline = compute_tour_length(x, tour_baseline)

        # L_tr and L_bl are tensors of shape (bsz,). Compute the mean tour length
        mean_tour_length_train += L_train.mean().item()
        mean_tour_length_baseline += L_baseline.mean().item()
    # 计算平均路径长度：通过累加并除以评估批次的数量来计算平均路径长度。
    mean_tour_length_train =  mean_tour_length_train/ args.nb_batch_eval
    mean_tour_length_baseline =  mean_tour_length_baseline/ args.nb_batch_eval

    # evaluate train model and baseline and update if train model is better
    # 条件判断：如果 model_train 的平均路径长度加上一个小容差 args.tol 小于 model_baseline 的平均路径长度，则更新基线模型。
    # 更新基线：将 model_train 的状态复制给 model_baseline，确保基线模型总是最优的。
    update_baseline = mean_tour_length_train+args.tol < mean_tour_length_baseline
    if update_baseline:
        model_baseline.load_state_dict( model_train.state_dict() )

    # Compute TSPs for small test set
    # Note : this can be removed
    # 在固定的测试集 x_1000tsp 上评估 model_baseline 的性能，计算平均路径长度。
    with torch.no_grad():
        tour_baseline, _ = model_baseline(x_1000tsp, deterministic=True)
    mean_tour_length_test = compute_tour_length(x_1000tsp, tour_baseline).mean().item()
    
    # For checkpoint
    #性能记录：将每个 epoch 的训练和基线模型的平均路径长度记录到列表中，方便后续绘图或分析。
    # 计算优化差距：根据节点数量计算训练模型相对于已知最优解的优化差距。
    # 日志输出：打印并写入日志文件，记录每个 epoch 的关键信息，如时间消耗、路径长度等。
    plot_performance_train.append([ (epoch+1), mean_tour_length_train])
    plot_performance_baseline.append([ (epoch+1), mean_tour_length_baseline])
        
    # Compute optimality gap
    if args.nb_nodes==50: gap_train = mean_tour_length_train/5.692- 1.0
    elif args.nb_nodes==100: gap_train = mean_tour_length_train/7.765- 1.0
    else: gap_train = -1.0
    
    # Print and save in txt file
    mystring_min = 'Epoch: {:d}, epoch time: {:.3f}min, tot time: {:.3f}day, L_train: {:.3f}, L_base: {:.3f}, L_test: {:.3f}, gap_train(%): {:.3f}, update: {}'.format(
        epoch, time_one_epoch/60, time_tot/86400, mean_tour_length_train, mean_tour_length_baseline, mean_tour_length_test, 100*gap_train, update_baseline) 
    print(mystring_min) # Comment if plot display
    file.write(mystring_min+'\n')
#     all_strings.append(mystring_min) # Uncomment if plot display
#     for string in all_strings: 
#         print(string)
    
    # Saving checkpoint
    # 创建检查点目录：确保存在 checkpoint 目录，如果没有则创建。
    # 保存检查点：使用 torch.save 将当前训练状态保存到文件中，包括 epoch 数、时间信息、损失值、模型状态字典、优化器状态等，以便后续恢复训练或评估。
    checkpoint_dir = os.path.join("checkpoint")
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    torch.save({
        'epoch': epoch,
        'time': time_one_epoch,
        'tot_time': time_tot,
        'loss': loss.item(),
        'TSP_length': [torch.mean(L_train).item(), torch.mean(L_baseline).item(), mean_tour_length_test],
        'plot_performance_train': plot_performance_train,
        'plot_performance_baseline': plot_performance_baseline,
        'mean_tour_length_test': mean_tour_length_test,
        'model_baseline': model_baseline.state_dict(),
        'model_train': model_train.state_dict(),
        'optimizer': optimizer.state_dict(),
        }, '{}.pkl'.format(checkpoint_dir + "/checkpoint_" + time_stamp + "-n{}".format(args.nb_nodes) + "-gpu{}".format(args.gpu_id)))
    
    # early_stopping(mean_tour_length_test, model_train)

    # if early_stopping.early_stop:
    #     print("Early stopping")
    #     break

KeyboardInterrupt: 